# Wasserstein Deep Convolutional Conditional Generative Adversarial Network (with Gradient Penalty) (WDCCGAN-GP)

A model skeleton for Wasserstein Deep Convolutional Generative Adversarial Networks (with Gradient Penalty) (WDCCGAN-GP)s. It was first introduced in the paper titled *[Improved Training of Wasserstein GANs](https://arxiv.org/abs/1704.00028)* by Ishaan Gulrajani, Faruk Ahmed, Martin Arjovsky, Vincent Dumoulin, Aaron Courville.

## usage

In [3]:
import deeply
import deeply.datasets as dd
from deeply.transformers.scaler import image_scaler

mnist, info = dd.load("mnist", shuffle_files = True, as_supervised = True, with_info = True)
image_shape = info.features["image"].shape
n_classes   = info.features["label"].num_classes

gan = deeply.hub("wdcgan-gp", input_shape = image_shape, n_classes = n_classes,
                 decoder_batch_norm = True, encoder_dropout_rate = 0.3,
                 encoder_layer_growth_rate = 2, scaler = image_scaler,
                 init_decoder_units = 256, decoder_layer_growth_rate = 0.5, kernel_size = 5,
                 decoder_strides  = [1, 2], final_activation = "tanh")

## example

In [4]:
import tensorflow as tf
from deeply.plots import imgplot as imgplt, mplt

from tqdm.auto import tqdm, trange

def mapper(image, label):
    return image_scaler.fit_transform(image), tf.one_hot(label, n_classes)

config = dict(batch_size = 256,
              epochs = 20)
batch_size = config["batch_size"]

data   = mnist["train"].map(mapper)
batch  = data.batch(batch_size)

In [5]:
from deeply.model.gan import save_img_samples

samples = gan.get_random_latent_vector(shape = (batch_size, None))

for e in trange(config["epochs"]):
    for b in tqdm(batch):
        history = gan.fit(b, batch_size = batch_size, checkpoint_path = "gan.h5d5")
    save_img_samples(gan, samples = samples, to_file = "cgan.png")

  0%|          | 0/20 [00:00<?, ?it/s]

2022-10-11 19:08:28.013929: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


  0%|          | 0/235 [00:00<?, ?it/s]

2022-10-11 19:08:28.244984: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


ValueError: in user code:

    File "/Users/achillesrasquinha/miniforge3/lib/python3.10/site-packages/keras/engine/training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "/Users/achillesrasquinha/miniforge3/lib/python3.10/site-packages/keras/engine/training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/achillesrasquinha/miniforge3/lib/python3.10/site-packages/keras/engine/training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "/Users/achillesrasquinha/dev/deeply/src/deeply/model/gan.py", line 183, in train_step
        loss_discriminator = loss_discriminator + self._compute_gradient_penalty(data_feed, generated_output) * self.grad_weight
    File "/Users/achillesrasquinha/dev/deeply/src/deeply/model/gan.py", line 97, in _compute_gradient_penalty
        diff  = fake_output - real_output

    ValueError: Dimensions must be equal